In [2]:
# Import Packages
import sys, os
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot
from scipy import stats
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
 LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
 MarkerCluster, Heatmap,SearchControl, 
 FullScreenControl, LayerGroup, LegendControl)
from ipywidgets import Text, HTML, widgets, interactive, HBox, VBox
from branca.colormap import linear
%matplotlib inline

# Size and Extent of Fairfax County's Municipal Separate Storm Sewer (MS4)

Fairfax County's MS4 permit requires the County to map the `MS4 service area` and each `MS4 outfall`. Wood has identified over 10,000 terminal outfalls in the County, of which over 7,000 are the County's responsibility. The drainage areas to each MS4 outfall are delineated using an automated approach. This approach uses a hydrologically conditioned Digital Elevation Model (DEM) that incorporates StormNet to enforce subsurface drainage networks. This process is revisited annually to incorporate updates to StormNet and is documented in further detail in a SOP.

**MS4 outfall**
: the terminal point of manmade infrastructure that discharges to a water of the state or water of the US. Manmade infrastructure includes earthen ditches maintained to convey stormwater flow. 

**<font color=green>All MS4 drainage areas</font>**
: All the area draining to each MS4 outfall.

**<font color=red>MS4 service area</font>**
: Area draining to each County MS4 outfall, _except_ for excluded lands such as:
- VPDES permittees
- Other MS4 permittees (ex. VDOT)
- Forested lands (Note: the County chose to _not exclude_ Forested Lands from the MS4 service area)

The map below shows three layers/geometries that the County can be evaluated on:
1. <font color=red>MS4 service area</font>
2. <font color=green>All MS4 drainage areas</font>
3. <font color=blue>The entire County</font>

In [3]:
Drainage_Areas = gpd.read_file("All_Drainage_Areas.geojson", driver="GeoJSON")
Drainage_Areas = Drainage_Areas.rename(columns={"STORMNET_I":"STORMNET_ID"})

In [4]:
Service_Areas = gpd.read_file("County_MS4_Service_Areas_Fea.geojson", driver="GeoJSON")
Service_Areas = Service_Areas.rename(columns={"STORMNET_I":"STORMNET_ID"})

In [5]:
County = gpd.read_file("https://opendata.arcgis.com/datasets/58cf8abd870e47aeb1be8911983d2d44_15.geojson")

In [6]:
center = (38.8554638, -77.2757340)
m = Map(center=center, zoom=10)

geodata_SA = GeoData(geo_dataframe = Service_Areas,
                  style={'color': 'black', 'fillColor': '#eb1b0c', 'opacity':0.03, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                         hover_style={'fillColor': '#b08a3e', 'fillOpacity': 0.8},
                                      name = 'MS4 Service Area')
geodata_DA = GeoData(geo_dataframe = Drainage_Areas,
                  style={'color': 'black', 'fillColor': '#2deb0c', 'opacity':0.03, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                         hover_style={'fillColor': '#b08a3e', 'fillOpacity': 0.8},
                                      name = 'All Drainage Areas')
geodata_County = GeoData(geo_dataframe = County,
                  style={'color': 'black', 'fillColor': '#053ef7', 'opacity':0.03, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.2},
                         hover_style={'fillColor': '#b08a3e', 'fillOpacity': 0.8},
                                      name = 'County')

m.add_layer(geodata_SA)
m.add_layer(geodata_DA)
m.add_layer(geodata_County)

search = SearchControl(position="topleft", 
                       url=
                       'https://nominatim.openstreetmap.org/search?format=json&q={s}',
                       zoom=14,
                       property_name='display_name'
                      )
m.add_control(search)
control = FullScreenControl()
m.add_control(control)
m.add_control(LayersControl())
legend = LegendControl({"MS4 Service Area":"#eb1b0c", "All Drainage Areas":"#2deb0c", "County":"#053ef7"}, name="Legend", position="bottomright")
m.add_control(legend)

m

Map(center=[38.8554638, -77.275734], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [7]:
MS4_Scenarios = pd.read_csv("MS4_Scenarios.csv")
MS4_Scenarios = MS4_Scenarios.iloc[0:3]

The chart below depicts three stacked bars. Each bar corresponds to a polygon geometry displayed in the map above. Within each geometry (ex. MS4 service area ~ current MS4) the area can be broken down into three categories:
1. Impervious Area
2. Pervious Area
3. Forest within the Pervious Area (this is a subset of #2)

*Note: Excluded Lands (other than forest) have been removed from each geometry*

In [8]:
MS4_Scenarios_AC = MS4_Scenarios[['Scenarios','Impervious Area', 'Pervious Area', 'Forest in Pervious Area']]
MS4_Scenarios_AC = pd.melt(MS4_Scenarios_AC, id_vars = ['Scenarios'])

In [9]:
fig_bar = px.bar(MS4_Scenarios_AC, x='Scenarios', y='value', color='variable', color_discrete_sequence =['red','blue','green'],
             labels={
                     "Scenarios": "MS4 Responsibility Scenarios",
                     "value": "Acres",
                     "variable": "Land Cover Type"
                 },
             title='MS4 With Excluded Lands Removed')

f1 = go.FigureWidget(fig_bar)
f1

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': ('Land Cover Type=Imperv…

In [17]:
MS4_Scenarios[['Scenarios','Impervious Area', 'Pervious Area', 'Forest in Pervious Area']]

,Scenarios,Impervious Area,Pervious Area,Forest in Pervious Area
0,Current MS4,"21,012.45","50,839.16","29,887.81"
1,Drainage To All County Outfalls Of Any Ownership,"42,867.46","77,572.75","44,194.92"
2,Entire County,"56,494.71","203,828.50","69,381.93"


The next chart displays the `Excluded Lands + Forest`. Each bar on this chart also corresponds with a geometry; however, the geometry is slightly different than before. Instead of using the MS4 service area for the "Current MS4", we use County drainage areas. The County drainage areas do not have excluded lands removed - such as the VDOT MS4/ROW. 

Each bar shows the excluded lands present within its geometry.

*Note: Gif below displays the difference between 4 relevant geometries - All Drainage Areas, County Drainage Areas, the MS4 Service Area, and the entire County.*

In [22]:
from IPython.display import Image
Image(url='https://media.giphy.com/media/XSICRHtziK2jKnlrwr/giphy.gif')

In [10]:
MS4_Scenarios_Removed = pd.read_csv("MS4_Scenarios_Removed.csv")

,Scenarios,VDOT,Other MS4s,VPDES Permitees,Total Forest
0,Current MS4,12973.30102,369.190364,311.178562,35837.63853
1,Drainage To All County Outfalls Of Any Ownership,18793.31833,1878.809257,970.206672,49777.84591
2,Entire County,24483.20377,4747.520370,3462.219218,142877.96330


In [11]:
MS4_Scenarios_Removed_melt = pd.melt(MS4_Scenarios_Removed, id_vars = ['Scenarios'], value_vars = ['VDOT', 'Other MS4s', 'VPDES Permitees', 'Total Forest'])

In [12]:
fig_bar1 = px.bar(MS4_Scenarios_Removed_melt, x='Scenarios', y='value', color='variable', color_discrete_sequence =['orange','blue','red','green'],
             labels={
                     "Scenarios": "MS4 Responsibility Scenarios",
                     "value": "Acres",
                     "variable": "Potentially Excluded Lands"
                 },
             title='Excluded Lands + Forest')

f2 = go.FigureWidget(fig_bar1)
f2

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': ('Potentially Excluded L…

In [ ]:
MS4_Scenarios_Removed

The County's MS4 permit will require it to reduce 100% of the L2 Scoping Run POC reductions required for existing sources as of June 30, 2009 by the end of its third permit cycle. Table 3b from the Chesapeake Bay TMDL Guidance can be used to estimate the scope of these required reductions within the Potomac River Basin. 

Impervious and pervious area estimates for the three geometries can be plugged into the column`Land served by MS4 (2009)` to estimate what reductions would be required under different MS4 size and extent scenarios. The results are shown in the interactive chart below. 

The dropdown menu allows the user to select different POC.

### Table 3b. Current MS4
#### Calculation Sheet for Estimating Existing Source Loads and Reduction Requirements for the Potomac River Basin

| Pollutant | Subsource | Loading rate (lbs/ac/yr) | Land served by MS4 (2009) | Load (lbs/yr) | L2 Required Reduction | 100% Cumulative Reduction Required (lbs/yr)
| :---------: | ----------- | :---------: | :---------: |  :---------: |  :---------: | :---------: |
| TN | Regulated impervious | 16.86 | 21,012.45 | 354,269.91 | 9% | 31,884.29
| TN | Regulated pervious |10.07 | 50,839.16 | 511,950.34 | 6% | 30,717.02
| TP | Regulated impervious | 1.62 | 21,012.45 | 34,040.17 | 16% | 5,446.43
| TP | Regulated pervious |0.41 | 50,839.16 | 20,844.06 | 7.25% | 1,511.19
| TSS | Regulated impervious | 1,171.32 | 21,012.45 | 24,612,302.93 | 20% | 4,922,460.59
| TSS | Regulated pervious |175.8 | 50,839.16 | 8,937,524.33| 7.25% | 782,033.38

In [13]:
MS4_Scenarios_POC = MS4_Scenarios[['Scenarios','TN', 'TP', 'TSS']]

,Scenarios,TN,TP,TSS
0,Current MS4,"62,601.31","6,957.62","5,704,493.97"
1,Drainage To All County Outfalls Of Any Ownership,"111,916.53","13,417.09","11,235,564.70"
2,Entire County,"208,878.25","20,702.23","16,370,068.97"


In [14]:
MS4_Scenarios_POC = MS4_Scenarios_POC.set_index(['Scenarios'])

In [15]:
fig = go.Figure()

for column in MS4_Scenarios_POC.columns.to_list():
    fig.add_trace(
        go.Bar(
            x = MS4_Scenarios_POC.index,
            y = MS4_Scenarios_POC[column],
            name = column
        )
    )
  
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True]},
                          {'title': 'All',
                           'showlegend':True}]),
            dict(label = 'TN',
                  method = 'update',
                  args = [{'visible': [True, False, False]},
                          {'title': 'TN',
                           'showlegend':True}]),
             dict(label = 'TP',
                  method = 'update',
                  args = [{'visible': [False, True, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'TP',
                           'showlegend':True}]),
             dict(label = 'TSS',
                  method = 'update',
                  args = [{'visible': [False, False, True]},
                          {'title': 'TSS',
                           'showlegend':True}]),
            ])
        )
    ])

f2 = go.FigureWidget(fig)
f2

#fig.show()

FigureWidget({
    'data': [{'name': 'TN',
              'type': 'bar',
              'uid': 'b54cdc81-2d46-44…

In [ ]:
MS4_Scenarios_POC